In [26]:
import networkx as nx



inputFile = "/Users/soheil.danesh/Documents/PROJECTS/Mediator Complex Functional Differences in Homologs/BIOGRID-ORGANISM-3.4.143.tab/BIOGRID-ORGANISM-Homo_sapiens-3.4.143.tab"
rdd = sc.textFile(inputFile)
rdd.take(5)

proteinRelationsTable = rdd.collect()

        
        
    
    

rdd = rdd.map(lambda row : row.split('\t') )

#rdd.coalesce(1).saveAsTextFile('/Users/soheil.danesh/Documents/PROJECTS/hubClique/biogrid-homoSapiens-ppi.csv')

proteinRelationsTable = rdd.collect()

for i in range(0, 10):
    print(proteinRelationsTable[i])

#read input file



[u'INTERACTOR_A', u'INTERACTOR_B', u'OFFICIAL_SYMBOL_A', u'OFFICIAL_SYMBOL_B', u'ALIASES_FOR_A', u'ALIASES_FOR_B', u'EXPERIMENTAL_SYSTEM', u'SOURCE', u'PUBMED_ID', u'ORGANISM_A_ID', u'ORGANISM_B_ID']
[u'ETG6416', u'ETG2318', u'MAP2K4', u'FLNC', u'JNKK|JNKK1|MAPKK4|MEK4|MKK4|PRKMK4|SAPKK-1|SAPKK1|SEK1|SERK1|SKK1', u'ABP-280|ABP280A|ABPA|ABPL|FLN2|MFM5|MPD4', u'Two-hybrid', u'Marti A (1997)', u'9006895', u'9606', u'9606']
[u'ETG84665', u'ETG88', u'MYPN', u'ACTN2', u'CMD1DD|CMH22|MYOP|RCM4', u'CMD1AA', u'Two-hybrid', u'Bang ML (2001)', u'11309420', u'9606', u'9606']
[u'ETG90', u'ETG2339', u'ACVR1', u'FNTA', u'ACTRI|ACVR1A|ACVRLK2|ALK2|FOP|SKR1|TSRI', u'FPTA|PGGT1A|PTAR2', u'Two-hybrid', u'Wang T (1996)', u'8599089', u'9606', u'9606']
[u'ETG2624', u'ETG5371', u'GATA2', u'PML', u'DCML|IMD21|MONOMAC|NFE1B', u'MYL|PP8675|RNF71|TRIM19', u'Two-hybrid', u'Tsuzuki S (2000)', u'10938104', u'9606', u'9606']
[u'RP4-547C9.3', u'ETG6774', u'RPA2', u'STAT3', u'REPA2|RP-A p32|RP-A p34|RPA32', u'ADMIO|AP

In [27]:
#make graph
#a node consists of an ngram(term) and its weight(Statistical)

def makeGraph(relationsTable, nodeColumns=[2,3]): #todo maybe later add a weight column

    if(len(nodeColumns) != 2):
        raise 'nodeColumns is supposed to have two members each being the index of the column in the table where the nodes of the graph are'

    G = nx.Graph()
    for row in relationsTable:
        node1 = row[nodeColumns[0]]
        node2 = row[nodeColumns[1]]
        G.add_edges_from([(node1,node2)])  
        
    return G
            

#for source med12

#find neightbors

#for each neighbor that has no weight assignments yet assign a new neigh

In [28]:
graph = makeGraph(proteinRelationsTable)

In [29]:
from collections import deque

def updateNodeWeightsMap( node, weight, nodeWeightsMap):
    firstTimeSeeingNode = True 
    if(node in nodeWeightsMap):
        nodeWeightsMap[node] += weight
        firstTimeSeeingNode = False
    else:
        nodeWeightsMap[node] = weight
        
    return firstTimeSeeingNode


#the way blast works is as the activation diffuses, we make sure each node emenates only once by adding only weights that are seeing for the first time to the sources queue and dequeueing after one emanation
#btw difusion happens by dividing actvation reached a node by the node's degree and sending it to its neightbor, with an optional decay factor also multiplied into each weight, 
def spreadingActivationSourceWeight(graph, sourceNodeName, weight=1.0, decayFactor=1.0):
    
    nodeWeightsMap = {}
    
    sourcesq = deque([ (sourceNodeName, weight) ])
    
    while(len(sourcesq)>0):
        nodeWeight = sourcesq.popleft()
        nodeName = nodeWeight[0]
        
        if(graph.degree(nodeName) == 0):
            raise 'node has degree of 0, develop this code branch'
            
        emenatedWeight = nodeWeight[1] * decayFactor / float(graph.degree(nodeName))
        for neighbor in graph[nodeName]:
            #add the emenated weight to the neighbor and also see whether it is the first time we are seeing this neighbor or not
            firstTimeSeeingNode = updateNodeWeightsMap(neighbor, emenatedWeight, nodeWeightsMap)
            if(firstTimeSeeingNode):# if it's the first time we are seeing this node make sure it emenates in the next iteration
                sourcesq.append( (neighbor, emenatedWeight) )
                
    #put node-weight pairs in nodeWeightMap into a sorted array as well
    nodeWeightsArr = []
    for node in nodeWeightsMap:
        nodeWeightsArr.append( (node, nodeWeightsMap[node]) )
        
    nodeWeightsArr = sorted(nodeWeightsArr, key=lambda nodeWeight: nodeWeight[1])
        
    return (nodeWeightsArr, nodeWeightsMap)


    #TODO?
    #def normalizeNodeWeightArr():
        
    
    
    


In [30]:
(nodeWeightsArr, nodeWeightsMap) = spreadingActivationSourceWeight(graph, 'MED12')
for nodeWeight in nodeWeightsArr:
    print(nodeWeight)

(u'ICOSLG', 7.132734551432847e-11)
(u'GRAMD1B', 1.4087834469703143e-10)
(u'ERAP2', 1.413842393387331e-10)
(u'GEMIN8P4', 2.2560078722546444e-10)
(u'COMP', 2.2560078722546444e-10)
(u'UPK1A', 2.348709608606709e-10)
(u'ARL17A', 2.7204800812482475e-10)
(u'LY6H', 2.841648662264907e-10)
(u'CLEC10A', 3.7567558585875044e-10)
(u'NCR2', 4.492842219432427e-10)
(u'ZCCHC24', 4.591462392764995e-10)
(u'CCDC167', 4.591462392764995e-10)
(u'KCNK17', 4.591462392764995e-10)
(u'SNORA12', 4.591462392764995e-10)
(u'APOC4', 4.591462392764995e-10)
(u'Hnf4a', 4.6359222888913894e-10)
(u'GUF1', 4.992914186002993e-10)
(u'HSP90B3P', 4.992914186002993e-10)
(u'ced-12', 5.635133787881257e-10)
(u'ced-5', 5.635133787881257e-10)
(u'TREM1', 5.990456292576569e-10)
(u'PTGES', 6.422011624659847e-10)
(u'FAM167A', 6.571672596032746e-10)
(u'PAX4', 6.571672596032746e-10)
(u'Tfrc', 6.702816304497169e-10)
(u'Sult1e1', 7.026549351567331e-10)
(u'Pmpca', 7.026549351567331e-10)
(u'Twf1', 7.026549351567331e-10)
(u'Trpc1', 7.026549351567

In [55]:
(nodeWeightsArr, nodeWeightsMap) = spreadingActivationSourceWeight(graph, 'MED12L')
def printNodeWeightsArr(nodeWeightsArr, limit=-1):
    if(limit == -1):
        limit = len(nodeWeightsArr)
        
    rank = 0
    for nodeWeight in nodeWeightsArr[0:limit]:
        rank += 1
        print("rank = "+ str(rank) + " nodeWeight = "+ str(nodeWeight))
printNodeWeightsArr(nodeWeightsArr,20)

rank = 1 nodeWeight = (u'Pgk2', 2.3096844974154655e-11)
rank = 2 nodeWeight = (u'Scg5', 2.3096844974154655e-11)
rank = 3 nodeWeight = (u'Psma3', 2.3096844974154655e-11)
rank = 4 nodeWeight = (u'Cope', 2.3096844974154655e-11)
rank = 5 nodeWeight = (u'Copg1', 2.3096844974154655e-11)
rank = 6 nodeWeight = (u'Sec16a', 2.3096844974154655e-11)
rank = 7 nodeWeight = (u'LOC100361858', 2.3096844974154655e-11)
rank = 8 nodeWeight = (u'RGD1566197', 2.3096844974154655e-11)
rank = 9 nodeWeight = (u'Snrnp27', 2.3096844974154655e-11)
rank = 10 nodeWeight = (u'Sncb', 2.3096844974154655e-11)
rank = 11 nodeWeight = (u'Vgf', 2.3096844974154655e-11)
rank = 12 nodeWeight = (u'Fam114a1', 2.3096844974154655e-11)
rank = 13 nodeWeight = (u'Fstl4', 2.3096844974154655e-11)
rank = 14 nodeWeight = (u'Grpel1', 2.3096844974154655e-11)
rank = 15 nodeWeight = (u'Lmnb2', 2.3096844974154655e-11)
rank = 16 nodeWeight = (u'Sec11a', 2.3096844974154655e-11)
rank = 17 nodeWeight = (u'Copb2', 2.3096844974154655e-11)
rank = 18

In [63]:


#vector normalization, addition, subtraciton and dot product using spark

#vector is a rdd of tuples like (u'CDK8', 0.13727614584604667) ie [(u'CDK8', 0.13727614584604667), .....]
    #bookmark: make these functions, then for thyroid cancer:
    #after normalizing vectors
    # add the vectors of the source nodes to find a list of likely candidate genese that may also be involved
    
    #for med complex subtract the vecotre of individual homolougs e.g. med12 - med12L

    

def normalizNodeWeightsArr(nodeWeightsArr):
    nodeWeightsRdd = sc.parallelize(nodeWeightsArr)
    sumNormalizedRdd = normalizNodeWeightsRdd(nodeWeightsRdd)
    sumNormalizedSortedNodeWeightsArray = sumNormalizedRdd.map(lambda (a,b) : (b,a)).sortByKey(ascending=False).map(lambda (a,b) : (b,a)).collect()
    return sumNormalizedSortedNodeWeightsArray

def normalizNodeWeightsRdd(nodeWeightsRdd):
    weightsSum = nodeWeightsRdd.map(lambda nodeWeight: nodeWeight[1]).sum()
    sumNormalizedRdd = nodeWeightsRdd.map(lambda nodeWeight: (nodeWeight[0], float(nodeWeight[1]) / float(weightsSum) )) 
    return sumNormalizedRdd

def normalizeAndAddNodeWeightArrs(nodeWeightsArrs):
    return normalizeAndOperateOnNodeWeightArrs(nodeWeightsArrs, operation='add')

def normalizeAndSubtractNodeWeightArrs(nodeWeightsArrs):
    return normalizeAndOperateOnNodeWeightArrs(nodeWeightsArrs, operation='subtract')


def normalizeAndOperateOnNodeWeightArrs(nodeWeightsArrs, operation='add'):
    resultsRdd = normalizNodeWeightsRdd(sc.parallelize(nodeWeightsArrs[0]))
    for arr in nodeWeightsArrs:
        resultsRdd.union(normalizNodeWeightsRdd(sc.parallelize(arr)))
        if(operation == 'add'):
            resultsRdd = resultsRdd.reduceByKey(lambda a,b: a+b)
        elif( operation == 'subtract'):
            resultsRdd = resultsRdd.reduceByKey(lambda a,b: a-b)
            
    
    sortedresultsArr = resultsRdd.map(lambda (a,b) : (b,a)).sortByKey(ascending=False).map(lambda (a,b) : (b,a)).collect() #from http://stackoverflow.com/questions/24656696/spark-get-collection-sorted-by-value/40708098#40708098 
    return sortedresultsArr
        
        
    
#def subtractVectors(): tood bookmark

    
    

In [64]:
(nodeWeightsArr_med12, nodeWeightsMap_med12) = spreadingActivationSourceWeight(graph, 'MED12')
(nodeWeightsArr_med12L, nodeWeightsMap_med12L) = spreadingActivationSourceWeight(graph, 'MED12L')

med12LMinusMed12 = normalizeAndSubtractNodeWeightArrs([nodeWeightsArr_med12L, nodeWeightsArr_med12])

printNodeWeightsArr(med12LMinusMed12,20)


rank = 1 nodeWeight = (u'CDK8', 0.05194217031179771)
rank = 2 nodeWeight = (u'MED19', 0.05167552707015373)
rank = 3 nodeWeight = (u'CCNC', 0.05142065811755072)
rank = 4 nodeWeight = (u'CDK19', 0.051279721231552375)
rank = 5 nodeWeight = (u'MED7', 0.051108923335542844)
rank = 6 nodeWeight = (u'CDC5L', 0.049825009440240736)
rank = 7 nodeWeight = (u'AR', 0.0487048890814553)
rank = 8 nodeWeight = (u'NCAPD2', 0.04824611297338745)
rank = 9 nodeWeight = (u'MED12L', 0.004915874664261784)
rank = 10 nodeWeight = (u'MED1', 0.004354630527566603)
rank = 11 nodeWeight = (u'MED4', 0.004296211897582316)
rank = 12 nodeWeight = (u'MED26', 0.004272633194561377)
rank = 13 nodeWeight = (u'MED24', 0.004249556610889812)
rank = 14 nodeWeight = (u'MED23', 0.004244517185294739)
rank = 15 nodeWeight = (u'MED10', 0.004233894961573892)
rank = 16 nodeWeight = (u'MED29', 0.004163560715324846)
rank = 17 nodeWeight = (u'MED12', 0.004098711756101212)
rank = 18 nodeWeight = (u'MED28', 0.004089955197728652)
rank = 19 nod

In [66]:
med12MinusMed12L = normalizeAndSubtractNodeWeightArrs([nodeWeightsArr_med12, nodeWeightsArr_med12L])
printNodeWeightsArr(med12MinusMed12L,50)

rank = 1 nodeWeight = (u'MED1', 0.009678870178640914)
rank = 2 nodeWeight = (u'MED17', 0.009147391677978355)
rank = 3 nodeWeight = (u'MED23', 0.009090120416902473)
rank = 4 nodeWeight = (u'MED4', 0.009014183658366369)
rank = 5 nodeWeight = (u'MED24', 0.008967718861779127)
rank = 6 nodeWeight = (u'MED16', 0.008908470399384746)
rank = 7 nodeWeight = (u'MED14', 0.008727766109434437)
rank = 8 nodeWeight = (u'MED26', 0.008552952471337813)
rank = 9 nodeWeight = (u'MED29', 0.008487597206907727)
rank = 10 nodeWeight = (u'CDK8', 0.008452762712172233)
rank = 11 nodeWeight = (u'MED28', 0.008384817195665218)
rank = 12 nodeWeight = (u'MED10', 0.0083128100076417)
rank = 13 nodeWeight = (u'FBXW7', 0.008290939590039782)
rank = 14 nodeWeight = (u'MED13', 0.008271005139313602)
rank = 15 nodeWeight = (u'MED7', 0.008234118228122407)
rank = 16 nodeWeight = (u'MED6', 0.00823393441626804)
rank = 17 nodeWeight = (u'MED15', 0.008185904723099878)
rank = 18 nodeWeight = (u'MED9', 0.008137407915557561)
rank = 19 

In [59]:
#sumNormalizedSortedNodeWeightsArray = normalizNodeWeightsArr(nodeWeightsArr)
#print(sumNormalizedSortedNodeWeightsArray)

#thyroid cancer genes
Text of email from Vala about genes involved in thyroid cancer,
eblow I've studied BRAF and TERT and show that the TERT promoters are all ranked in the top 50 or so ranked genes. also RAF1 seems involved in thyroid cancer in a 2015 paper:
miR-195 is a key regulator of Raf1 in thyroid cancer
(https://www.ncbi.nlm.nih.gov/pubmed/26527888)

BRAF mutation co- occurring with a TERT promoter, PIK3CA, TP53, or AKT1 mutation (155–157,577). Such a combination of several mutations is seen in a much smaller fraction of PTC as compared with a 40%–45% incidence of BRAF mutations and is expected to serve as a more specific marker of unfa- vorable outcomes of PTC.

The combination of a TERT mutation and a BRAF mutation within the same tumor was associated with a high risk of structural disease recurrence 

seven-gene mutational panel (BRAF, RAS, PAX8/PPARc, RET/PTC) would be more likely to demonstrate similar performance to that of a nonpregnant population.

Mutational testing for BRAF in AUS/FLUS samples has high specificity for cancer, but low sensitivity (198,199). Testing for a panel of mutations (BRAF, NRAS, HRAS, KRAS, RET/PTC1, RET/PTC3, PAX8/PPARc) offers a significantly higher sensitivity of 63%–80% ( 


In [ ]:
#thyroid cancer genes

(braf_nodeWeightsArr, braf_nodeWeightsMap) = spreadingActivationSourceWeight(graph, 'BRAF')
#print(braf_nodeWeightsArr)
(tert_nodeWeightsArr, tert_nodeWeightsMap) = spreadingActivationSourceWeight(graph, 'TERT')

braf_and_tert_nodeWeightArr = normalizeAndAddNodeWeightArrs([braf_nodeWeightsArr, tert_nodeWeightsArr])

    


In [56]:
printNodeWeightsArr(braf_and_tert_nodeWeightArr, 50)


rank = 1 nodeWeight = (u'BRAF', 0.016802510285163406)
rank = 2 nodeWeight = (u'RAF1', 0.013123612653710608)
rank = 3 nodeWeight = (u'HSP90AA1', 0.01185316243447596)
rank = 4 nodeWeight = (u'YWHAB', 0.01165937489175415)
rank = 5 nodeWeight = (u'YWHAE', 0.011545815297543156)
rank = 6 nodeWeight = (u'BRAP', 0.011541908546638124)
rank = 7 nodeWeight = (u'MAP2K1', 0.011427799589455074)
rank = 8 nodeWeight = (u'YWHAZ', 0.011125551244745124)
rank = 9 nodeWeight = (u'NEDD4L', 0.010811718204515269)
rank = 10 nodeWeight = (u'MAPK1', 0.010708692936274699)
rank = 11 nodeWeight = (u'MAPK3', 0.010660165445596923)
rank = 12 nodeWeight = (u'AKT1', 0.010656014985907358)
rank = 13 nodeWeight = (u'FBXW7', 0.010652040530111588)
rank = 14 nodeWeight = (u'YWHAQ', 0.010627435040730234)
rank = 15 nodeWeight = (u'YWHAG', 0.010626629947289244)
rank = 16 nodeWeight = (u'ARAF', 0.010541333668828133)
rank = 17 nodeWeight = (u'YWHAH', 0.010525887092771528)
rank = 18 nodeWeight = (u'RAP1A', 0.01042261385409363)
rank

In [ ]:

#This code was done while experimenting to make the method 
#simple diffusion
#rules: each source starts with a labled weight score of 1. The lable could be the protin itself (To find related proteins) or a known funciton of it (to assign functions to proteins)
from collections import deque


sourcesq = deque([ ('MED12', 1) ])
nodeWeightsMap = {}
decayFactor = 0.95

def addToNodeWeightsMap( node, weight ):
    firstTimeSeeingNode = True 
    if(nodeWeight[0] in nodeWeightsMap):
        nodeWeightsMap[nodeWeight[0]] += nodeWeight[1]
        firstTimeSeeingNode = False
    else:
        nodeWeightsMap[nodeWeight[0]] = nodeWeight[1]
        
    return firstTimeSeeingNode


while(len(sourcesq)>0):
    nodeWeight = sourcesq.popleft()
    nodeName = nodeWeight[0] 
    nodesWeight = nodeWeight[1] 
    for neighbor in graph[nodeName]:
        firstTimeSeeingNode = addToNodeWeightsMap(nodeName, nodesWeight*decayFactor)
        if(firstTimeSeeingNode):
            sourcesq.append(nodeName, nodesWeight*decayFactor)
        
    



sources = [('MED12', 1)]
decayFactor = 0.95

nodeWeights = {} #keeps track of the list of nodes that have already emenated their activaiton
alreadyEmenated = [] #holds nodes that have already emenated their weights, a node can only emenate weights once for each weight label, to avoid flowback and oscilations



for nodeWeight in sourcesq:
    #alreadyEmenated.append(node)
    for neighbor in graph[nodeWeight[0]]:
        #if(neighbor not in alreadyEmenated)
            if(neighbor in nodeWeights):
                nodeWeights[neighbor] += nodeWeight[0] * decayFactor 
            else:
                nodeWeights[neighbor] = nodeWeight[0] * decayFactor 
        
    

In [12]:
print(graph['MED12'])

{u'MEN1': {}, u'PRPF8': {}, u'SNRNP200': {}, u'GLI3': {}, u'NTRK1': {}, u'THRA': {}, u'COX20': {}, u'ZC3H13': {}, u'SPTLC1': {}, u'BRD4': {}, u'CCDC82': {}, u'OBFC1': {}, u'NR1I3': {}, u'PTEN': {}, u'CCNC': {}, u'TRIP4': {}, u'CFTR': {}, u'MED29': {}, u'MED28': {}, u'VDR': {}, u'MED21': {}, u'MED20': {}, u'MED23': {}, u'MED22': {}, u'MED25': {}, u'MED24': {}, u'MED27': {}, u'MED26': {}, u'MED1': {}, u'MED6': {}, u'MED7': {}, u'MED4': {}, u'MED8': {}, u'MED9': {}, u'XPO1': {}, u'TADA2A': {}, u'PPARGC1A': {}, u'EFTUD2': {}, u'LZTS2': {}, u'SIRT7': {}, u'SIRT6': {}, u'TRA': {}, u'MED30': {}, u'MED31': {}, u'TRRAP': {}, u'POLR2E': {}, u'EHMT2': {}, u'HIF1A': {}, u'EPAS1': {}, u'ESR2': {}, u'ESR1': {}, u'MED13L': {}, u'HECW2': {}, u'MYO1C': {}, u'RCOR1': {}, u'TAF15': {}, u'ECD': {}, u'NANOG': {}, u'Junb': {}, u'LYST': {}, u'FBXW7': {}, u'Bag2': {}, u'KLHL12': {}, u'KAT2A': {}, u'SREBF1': {}, u'CDK8': {}, u'CDK9': {}, u'TADA3': {}, u'SOX9': {}, u'QKI': {}, u'CDK19': {}, u'SUPT3H': {}, u'MED

In [20]:
import community
parts = community.best_partition(graph)
#print(part)
#print("-------")

#parts is an array where keys are nodes, values are paritions
partitions = {}
for node in parts: 
    partition = parts[node]
    if( partition in partitions):
        partitions[partition].append(node)
    else:
        partitions[partition] = [node]
        
for key in partitions:
    print("-------------")
    print(partitions[key])
    print("-------------")
        

-------------
[u'UBE2Q1', u'PHM8', u'ARC35', u'SPPL2A', u'cul1', u'AT5G53140', u'BCL2A1', u'ITGAM', u'BCY1', u'SYP1', u'CADPS2', u'STP2', u'DLD3', u'SPHK1', u'ATG4B', u'ATG4A', u'ATG4D', u'AT1G79630', u'Flna', u'ISW1', u'TMC8', u'ERG6', u'OGFR', u'RPC40', u'FAM71F1', u'SPG20', u'AMS1', u'RNF133', u'RNF138', u'TSHZ2', u'HMO1', u'CTPS2', u'HN1', u'NMB', u'TRNT1', u'ATG101', u'Cacna1b', u'TAX1BP1', u'ALPP', u'PDK4', u'ZFAND6', u'ZFAND5', u'ZFAND3', u'SUN5', u'SMAD6', u'Fbxl20', u'BMP6', u'BMP4', u'BMP2', u'TTC1', u'GDA1', u'MET6', u'AVP', u'BTG4', u'C10orf90', u'ADH1', u'FHOD3', u'ADH7', u'Rrh', u'R3HDM4', u'TRIML1', u'TMEM160', u'Tssk5', u'Aacs', u'YOR292C', u'VPS64', u'TP53I3', u'TRIM8', u'ZFYVE20', u'ITCH', u'AT4G28400', u'Trim30a', u'TVP15', u'TUFT1', u'RHO', u'Rnf25', u'TRIM5', u'NKP1', u'DRC7', u'OXT', u'GPP1', u'TBC1D1', u'HYPM', u'HYPK', u'Dusp10', u'MCL1', u'ATG16L1', u'FKBP10', u'PLA2G7', u'Usp7', u'APM2', u'CSHL1', u'UNC5A', u'Tbk1', u'RPL18B', u'Rptor', u'ACSS3', u'Cntfr', u'M

In [21]:
graph['MED12']

{u'BMI1': {},
 u'BRD4': {},
 u'Bag2': {},
 u'CASC3': {},
 u'CCDC82': {},
 u'CCNC': {},
 u'CDK19': {},
 u'CDK8': {},
 u'CDK9': {},
 u'CFTR': {},
 u'COX20': {},
 u'ECD': {},
 u'EFTUD2': {},
 u'EHMT2': {},
 u'EPAS1': {},
 u'ESR1': {},
 u'ESR2': {},
 u'FBXW7': {},
 u'GLI3': {},
 u'HECW2': {},
 u'HIF1A': {},
 u'Junb': {},
 u'KAT2A': {},
 u'KLHL12': {},
 u'LYST': {},
 u'LZTS2': {},
 u'MED1': {},
 u'MED10': {},
 u'MED11': {},
 u'MED13': {},
 u'MED13L': {},
 u'MED14': {},
 u'MED15': {},
 u'MED16': {},
 u'MED17': {},
 u'MED18': {},
 u'MED19': {},
 u'MED20': {},
 u'MED21': {},
 u'MED22': {},
 u'MED23': {},
 u'MED24': {},
 u'MED25': {},
 u'MED26': {},
 u'MED27': {},
 u'MED28': {},
 u'MED29': {},
 u'MED30': {},
 u'MED31': {},
 u'MED4': {},
 u'MED6': {},
 u'MED7': {},
 u'MED8': {},
 u'MED9': {},
 u'MEN1': {},
 u'MYO1C': {},
 u'Med22': {},
 u'NANOG': {},
 u'NR1I3': {},
 u'NTRK1': {},
 u'OBFC1': {},
 u'POLR2E': {},
 u'PPARGC1A': {},
 u'PRPF8': {},
 u'PTEN': {},
 u'QKI': {},
 u'RCOR1': {},
 u'SEC61B':

In [22]:
graph['MED12L']

{u'AR': {},
 u'CCNC': {},
 u'CDC5L': {},
 u'CDK19': {},
 u'CDK8': {},
 u'MED19': {},
 u'MED7': {},
 u'NCAPD2': {}}